In [4]:
import importlib

In [5]:
distance_count = 0
def l2_distance(a, b):
    global distance_count
    distance_count+=1
    return np.linalg.norm(a - b)

In [6]:
import hnsw
importlib.reload(hnsw)
from hnsw import HNSW
from hnsw import heuristic
import numpy as np
from datasets import load_sift_dataset, calculate_recall

In [12]:
hnsw_a = HNSW( distance_func=l2_distance, m=5, m0=7, ef=10, ef_construction=30,  neighborhood_construction = heuristic)
hnsw_b = HNSW( distance_func=l2_distance, m=5, m0=7, ef=10, ef_construction=30,  neighborhood_construction = heuristic)

In [13]:
hnsw_a.load('save/sift1m/hnsw_a.txt')
hnsw_b.load('save/sift1m/hnsw_b.txt')

In [14]:
merged_data = hnsw_a.data.copy()
merged_data.update(hnsw_b.data)

In [15]:
import merge_hnsw
importlib.reload(merge_hnsw)
from merge_hnsw import hnsw_general_merge
from merge_hnsw import layer_merge1_func

In [16]:
%%time
distance_count = 0
hnsw_merged1 = hnsw_general_merge(hnsw_a, hnsw_b, merged_data, layer_merge1_func)

Merging level: 0


Merging progress: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500000/500000 [31:08<00:00, 267.64it/s]


Merging level: 1


Merging progress: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 31358/31358 [01:09<00:00, 452.23it/s]


Merging level: 2


...

Merging level: 3


Merging progress: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 124/124 [00:00<00:00, 779.26it/s]


Merging level: 4


Merging progress: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 2759.71it/s]

CPU times: user 1h 4min 36s, sys: 10min 29s, total: 1h 15min 5s
Wall time: 1h 6min 16s


In [18]:
hnsw_merged1.save('save/sift1m/hnsw_merged1.txt')

In [17]:
print(f'Numer of distance calculated during merge: {distance_count}') # 696869746

696869746


In [ ]:
print('Merging accuracy', calculate_recall(hnsw_merged_naive, test_data, groundtruth=groundtruth_data, k=5, ef=1))

In [10]:
_, test_data, groundtruth_data = load_sift_dataset(train_file = None,
                                                      test_file='../datasets/sift1m-128d/sift_query.fvecs',
                                                      groundtruth_file='../datasets/sift1m-128d/sift_groundtruth.ivecs')

In [20]:
print('Merging accuracy', calculate_recall(hnsw_merged1, test_data, groundtruth=groundtruth_data, k=5, ef=32))

Calculating recall...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [01:08<00:00, 146.42it/s]

Merging accuracy (np.float64(0.8379), 487.7682)


In [21]:
print('Merging accuracy', calculate_recall(hnsw_merged1, test_data, groundtruth=groundtruth_data, k=5, ef=40))

Calculating recall...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [01:30<00:00, 109.93it/s]

Merging accuracy (np.float64(0.8687799999999999), 583.2042)


In [22]:
print('Merging accuracy', calculate_recall(hnsw_merged1, test_data, groundtruth=groundtruth_data, k=5, ef=64))

Calculating recall...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [02:49<00:00, 58.86it/s]

Merging accuracy (np.float64(0.91922), 862.3079)


In [11]:
search_ef = 32
distance_count = 0
recall, _ = calculate_recall(hnsw_merged1, test_data, groundtruth=groundtruth_data, k=5, ef=search_ef)
print(f'{search_ef},{recall}, {distance_count/len(test_data) }') 

Calculating recall...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [01:14<00:00, 133.67it/s]

32,0.83794, 609.8437


In [12]:
search_ef = 40
distance_count = 0
recall, _ = calculate_recall(hnsw_merged1, test_data, groundtruth=groundtruth_data, k=5, ef=search_ef)
print(f'{search_ef},{recall}, {distance_count/len(test_data) }') 

Calculating recall...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [01:29<00:00, 111.55it/s]

40,0.8689399999999999, 705.2153


In [13]:
search_ef = 64
distance_count = 0
recall, _ = calculate_recall(hnsw_merged1, test_data, groundtruth=groundtruth_data, k=5, ef=search_ef)
print(f'{search_ef},{recall}, {distance_count/len(test_data) }') 

Calculating recall...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [02:46<00:00, 60.15it/s]

64,0.9193399999999999, 984.2736
